In [ ]:
from math import tau

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import scipy.linalg as LA
import scipy.signal as ss

from utils import *

plt.style.use('seaborn-notebook')
np.random.seed(293710966)

# Esempio 3.14

In [ ]:
magnitudes = pd.read_csv('./harmonic-voltage-magnitude.csv', index_col='number')['typical']
phases = np.radians(pd.read_csv('./phases.csv', index_col='number'))['3.9']
dampings = pd.read_csv('./scaled-damping-factors.csv', index_col='number')['3.14']

power_freq = 50
sampling_freq = 2400
harmonic_numbers = np.arange(3, 14, 2)
no_of_harmonics = harmonic_numbers.max()

time = np.arange(4096.)

noise = np.random.normal(0, 0.1, time.size)
signal = noise.copy()

for n in harmonic_numbers:
    amp = magnitudes[n]
    phase = phases[n]
    damp = dampings[n] / sampling_freq
    omega = tau * n * power_freq / sampling_freq
    signal += amp * np.exp(-damp * time) * np.cos(omega * time + phase)

fig, ax = plt.subplots()
ax.plot(time[:200], signal[:200])
ax.set_xlabel('time [s]')
ax.set_ylabel('signal [V]')
fig.tight_layout()

In [ ]:
fastest_period = np.around(sampling_freq / power_freq).astype(int)
time_window = fastest_period * harmonic_numbers.max() * 2
data_size = signal.size - time_window + 1

windows = [signal[i : i + time_window] for i in range(data_size)]
data_matrix = np.vstack(windows)

print(f'numero di finestre: {data_matrix.shape[0]}')
print(f'larghezza finestre: {data_matrix.shape[1]}')

## Matrice delle Basi Ortogonali
La matrice $U$ viene ricavata mediante Decomposizione ai Valori Singolari (SVD) della matrice dei campionamenti $R_v$

In [ ]:
unitary_left, singuar_values, unitary_right = LA.svd(data_matrix.T, full_matrices=False)
unitary_left.shape, singuar_values.shape, unitary_right.shape

In [ ]:
sigma = np.diag(singuar_values)
np.allclose(unitary_left @ sigma @ unitary_right, data_matrix.T)

In [ ]:
unitary_signal = unitary_right[:, :no_of_harmonics]
U1 = unitary_signal[:time_window-1]
U2 = unitary_signal[-time_window+1:]
U1.shape, U2.shape

La matrice $U$ viene poi partizionata estraendo le prime e ultime $\left( M - 1 \right)$ colonne.

La matrice $U$ viene poi partizionata estraendo le prime e ultime $\left( M - 1 \right)$ colonne.

## Stima per Minimi Quadrati (LS) di $\Psi$
$$
\hat{\Psi}_{LS} = \left( U_1^H U_1 \right)^{-1} U_1^H U_2
$$

In [ ]:
psi_ls = LA.inv(U1.conj().T @ U1) @ U1.conj().T @ U2
psi_ls.shape

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.imshow((U1 @ psi_ls)[:50])
ax2.imshow(U2[:50])

fig.tight_layout()

## Stima delle Frequenze Armoniche
Calcolo degli autovalori di $\Psi$ coincidenti con i versori rotanti associati alle frequenze armoniche.

In [ ]:
eigvals = LA.eigvals(psi_ls)

some_eigvals = eigvals[eigvals.imag > 0]

fig, ax = plt.subplots()
plotzero(ax, eigvals.real, eigvals.imag, 'o')
plotzero(ax, some_eigvals.real, some_eigvals.imag, 'o')
fig.tight_layout()

In [ ]:
est_omegas = np.sort(np.angle(some_eigvals))
est_damps = -np.log(some_eigvals).real
est_freqs = est_omegas * sampling_freq / tau

fig, ax = plt.subplots()
ax.plot(power_freq * harmonic_numbers, 'o-', label='True Frequencies')
ax.plot(est_freqs, 'o-', label='Estimated Frequencies')
ax.set_ylabel('Frequency [Hz]')
ax.legend()
fig.tight_layout()

In [ ]:
est_damps